# 3rd Homework
## András Simon
---

In [1]:
from google.colab import drive
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
import matplotlib.pyplot as plt

I worked in Google Colab, so i had to upload my data. I used Google Drive to menage this.

I first stored the data on my computer in budapest_wheather_data.csv, then created the data.npy file with data_prepocessing.py. Then i uploaded the data.npy file to Google Drive and then loaded it to Colab.

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


The data is downloaded from http://idojarasbudapest.hu/archivalt-idojaras <br>
It contains the following months: <br>
2020 June <br>
2020 July <br>
2020 August <br>
2020 September <br>
2020 Oktober (the last sample is the 25th day)

The colums are the following: temp_max (celsius), temp_min (celsius), wind (m/s), rainfall (mm) <br>

In [3]:
data = np.load('/content/drive/My Drive/Data/data.npy')

data[:3]

array([[22.4, 12.1,  4.3,  1.5],
       [19.9, 13.5,  4.8,  0.2],
       [21.8, 11.7,  3.9,  1.3]])

Using only the average temperature

In [4]:
temp_data = (data[:, 0] + data[:, 1]) / 2

temp_data[:3]

array([17.25, 16.7 , 16.75])

Define the model

In [5]:
def make_1d_convnet(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=4):
    model = Sequential()

    model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)))
    model.add(MaxPooling1D())

    model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu'))
    model.add(MaxPooling1D())

    model.add(Flatten())
    model.add(Dense(nb_outputs, activation='linear'))

    model.compile(loss='mse', optimizer='adam', metrics=['mae'])

    return model

Using the data processing and the evaluating fuctions from the lesson <br> https://github.com/BME-SmartLab-Education/vitmav45/blob/master/07/1D-CNN-regression.ipynb

In [6]:
def make_timeseries_instances(timeseries, window_size):
    timeseries = np.asarray(timeseries)

    assert 0 < window_size < timeseries.shape[0] , "Out of range 0 < {} < {} ".format(window_size,timeseries.shape[0])

    X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
    Y = timeseries[window_size:]  # the target is the average temperature

    return X, Y

In [7]:
def evaluate_timeseries(timeseries, window_size, valid_split=0.15, test_split=0.15):
    filter_length = 5
    nb_filter = 4

    timeseries = np.atleast_2d(timeseries)

    if timeseries.shape[0] == 1:
        timeseries = timeseries.T       # 1D vektor -> 2D matrix

    nb_samples, nb_series = timeseries.shape

    model = make_1d_convnet(window_size=window_size, filter_length=filter_length, nb_input_series=nb_series, nb_outputs=nb_series, nb_filter=nb_filter)
    model.summary()

    X, Y = make_timeseries_instances(timeseries, window_size)
    
    valid_size = int(nb_samples*(1-test_split-valid_split))
    test_size = int(nb_samples*(1-test_split))
    X_train, Y_train = X[:valid_size], Y[:valid_size]
    X_valid, Y_valid = X[valid_size:test_size], Y[valid_size:test_size]
    X_test, Y_test   = X[test_size:], Y[test_size:]
    
    model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_valid, Y_valid), verbose=2)
    
    preds = model.predict(X_test)

    return Y_test, preds, model  # output the model to make forecast

Create and fit the model

In [8]:
targets, preds, model = evaluate_timeseries(temp_data, window_size=20, valid_split=0.20, test_split=0.20)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 16, 4)             24        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 4)              0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 4)              84        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2, 4)              0         
_________________________________________________________________
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 9         
Total params: 117
Trainable params: 117
Non-trainable params: 0
__________________________________________________________

Creating a forecaster

In [9]:
def forecast(model, timeseries, number_of_days):
    assert len(timeseries) > 20, "The length of timeseries should be larger then the window size (20) of the model"

    w = np.array([make_timeseries_instances(timeseries, 20)[0][-1, ...]])  # the last window

    preds = []

    for _ in range(number_of_days):
        pred = model.predict(w)  # get new day prediction
        preds.append(pred)
        w = np.array([np.concatenate([w[0, 1:, ...], pred], axis=0)])  # push the window towards the future

    return preds

In [10]:
preds = forecast(model=model, timeseries=temp_data, number_of_days=30)

preds

[array([[17.163973]], dtype=float32),
 array([[15.22974]], dtype=float32),
 array([[13.369744]], dtype=float32),
 array([[10.25327]], dtype=float32),
 array([[8.260723]], dtype=float32),
 array([[8.51174]], dtype=float32),
 array([[9.112501]], dtype=float32),
 array([[10.581965]], dtype=float32),
 array([[11.997331]], dtype=float32),
 array([[13.041879]], dtype=float32),
 array([[13.556527]], dtype=float32),
 array([[14.357994]], dtype=float32),
 array([[14.673014]], dtype=float32),
 array([[15.713849]], dtype=float32),
 array([[16.312702]], dtype=float32),
 array([[16.617083]], dtype=float32),
 array([[14.425499]], dtype=float32),
 array([[11.953587]], dtype=float32),
 array([[9.073668]], dtype=float32),
 array([[6.9516306]], dtype=float32),
 array([[8.255027]], dtype=float32),
 array([[10.77294]], dtype=float32),
 array([[12.621778]], dtype=float32),
 array([[14.156595]], dtype=float32),
 array([[15.011261]], dtype=float32),
 array([[15.424967]], dtype=float32),
 array([[16.109367]],

The last day in the temp_data is 2020 okt. 25, so we need: <br>
okt. 28 : 3rd pred <br>
nov. 3. : 10th pred <br>
nov. 24 : 30th pred <br>

In [11]:
print("Temperature forecast (celsius)\n")
print(f"Okt. 28 : {preds[2][0, 0]}\nnov.  3 : {preds[10][0, 0]}\nnov. 24 : {preds[29][0, 0]}")

Temperature forecast (celsius)

Okt. 28 : 13.369744300842285
nov.  3 : 13.556527137756348
nov. 24 : 16.718976974487305
